In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd       
train = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
train.shape

In [ ]:
train.columns.values

Посмотрим на пример

In [ ]:
print (train["review"][0])

Надо убрать HTML-теги

In [ ]:
from bs4 import BeautifulSoup                 
example1 = BeautifulSoup(train["review"][0])  
print (train["review"][0])
print (example1.get_text())

In [ ]:
import re
#Оставим в тексте только буквы
letters_only = re.sub("[^a-zA-Z]",           # Что искать
                      " ",                   # Чем заменить
                      example1.get_text() )  # Текст для поиска
print (letters_only)

In [ ]:
lower_case = letters_only.lower()  
#Разобъем текст на слова
words = lower_case.split()               

In [ ]:
#import nlt

Теперь надо разобраться со словами, которые не привносят особого смысла в текст (например предлоги).

In [ ]:
import nltk
from nltk.corpus import stopwords 
print(stopwords.words("english"))

In [ ]:
words = [w for w in words if not w in stopwords.words("english")]
print(words)

In [ ]:
def review_to_words( raw_review ):
    # Удаляем html-разметку
    review_text = BeautifulSoup(raw_review).get_text() 
    # Оставляем только буквы      
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # Переводим в нижний регистр и разбиваем на слова
    words = letters_only.lower().split()                             
    #Переведем в set
    stops = set(stopwords.words("english"))                  
    # Удаляем ненужные слова
    meaningful_words = [w for w in words if not w in stops]   
    #Все слова в один текст
    return( " ".join( meaningful_words ))

In [ ]:
clean_review = review_to_words( train["review"][0] )
print(clean_review)

In [ ]:
num_reviews = train["review"].size
clean_train_reviews = []
for i in range( 0, num_reviews ):
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

In [ ]:
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews )  )                                                                  
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Перейдем к числовому представлению текстов: соберем слова из текстов в словарь, построим вектор, который показывает количество вхождений слов из словаря в текст.

In [ ]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

In [ ]:
print(train_data_features.shape)

In [ ]:
vocab = vectorizer.get_feature_names()
print(vocab)

Посмотрим на количество вхождений в текст слов

In [ ]:
import numpy as np
dist = np.sum(train_data_features, axis=0)
for tag, count in zip(vocab, dist):
    print(count, tag)

RandomForestClassifier из 100 деревьев

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["sentiment"] )

In [ ]:
test = pd.read_csv("/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", \
                   quoting=3 )

print(test.shape)
num_reviews = len(test["review"])
clean_test_reviews = [] 

print ("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )